# Salesforce Optimization

## Set-up:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import sklearn.linear_model as sk
from sklearn.linear_model import LinearRegression as LR
import scipy.optimize as optimize
from scipy.optimize import curve_fit
from gurobipy import *

## Part A:

Below, we will compute the 4 Adbudg parameters for each of the eight drugs. First, we must set up the data vectors in order to get started.

In [3]:
proxy_for_infinity = 10
sales_force = [0, 0.5, 1.0, 1.5, proxy_for_infinity]

Next, let's set up our sales response values for each drug.

In [4]:
xl_file = pd.ExcelFile('delphi-consensus-outputs.xlsx')
sales_response = xl_file.parse("Sheet1")
sales_response = sales_response.drop(sales_response.columns[0], axis=1)
sales_response

,Naprosyn,Nipro,Anaprox,Norinyl,Pironil,Lidex,Synalar,Nasalide
0,0.30,0.47,0.15,0.31,0.45,0.56,0.59,0.15
1,0.45,0.68,0.48,0.63,0.70,0.80,0.76,0.61
2,1.00,1.05,1.04,1.03,1.01,1.02,1.02,1.07
3,1.40,1.26,1.20,1.15,1.05,1.11,1.07,1.46
4,1.60,1.52,1.35,1.25,1.10,1.20,1.11,1.76


Now, we can use nonlinear regression to predicut y as a function of x and our four unknown parameters (c, d, adbudg_min, adbudg_max). First we will build our function that represents the Adbudg model.

In [5]:
def compute_adbudg(x, c, d, adbudg_min, adbudg_max):
    return adbudg_min + (adbudg_max - adbudg_min) * (x**c) / (d + x**c)

Next, we will use `scipy.optimize.curve_fit` to predict our unknown parameters for each of the eight drugs.

In [6]:
# Initialize array for parameters of each of the eight drugs
adbudg_params = np.zeros((8, 4))

# Iterate through each drug and perform nonlinear regression to get the parameter estimates
for i in range(8):
    opt, cov = curve_fit(f=compute_adbudg, xdata=sales_force, ydata=sales_response.iloc[:,i], p0=(1, 1, sales_response.iloc[0][i], sales_response.iloc[4][i]))
    c_opt, d_opt, min_opt, max_opt = opt
    adbudg_params[i][0] = c_opt
    adbudg_params[i][1] = d_opt
    adbudg_params[i][2] = min_opt
    adbudg_params[i][3] = max_opt

df_drug_params = pd.DataFrame(adbudg_params, columns = ['c','d','adbudg_min','adbudg_max'], index = ['Naprosyn','Nipro','Anaprox','Norinyl','Pironil','Lidex','Synalar','Nasalide'])
df_drug_params

,c,d,adbudg_min,adbudg_max
Naprosyn,3.462680,0.841761,0.309324,1.606966
Nipro,2.264449,0.825955,0.469389,1.524101
Anaprox,2.817291,0.361170,0.148199,1.342579
Norinyl,2.618402,0.311903,0.309580,1.248045
Pironil,3.254370,0.162404,0.449586,1.091816
Lidex,2.089322,0.394394,0.560039,1.202236
Synalar,3.245910,0.211963,0.589636,1.105797
Nasalide,2.045422,0.676693,0.156967,1.781163


## Part B:

Next, let's compute the profit-maximizing number of salespersons for each of the eight drugs. We will be using the four adbudg parameters we found for each of the eight drugs in the previous part along with the current salesforce size, sales volume, and profit margin which we will load now.

In [7]:
xl_file2 = pd.ExcelFile('margin-revenue-salesforce.xlsx')
margin_sf = xl_file2.parse("drugs")
margin_sf = margin_sf.drop(margin_sf.columns[0], axis=1)
margin_sf = margin_sf.rename(index={0: 'Profit Margin', 1: 'Current/Original Revenue', 2: 'Current/Original Salesforce'})
margin_sf

,Naprosyn,Nipro,Anaprox,Norinyl,Pironil,Lidex,Synalar,Nasalide
Profit Margin,0.7,0.8,0.55,0.72,0.72,0.62,0.53,0.52
Current/Original Revenue,214.4,210.1,36.50,21.20,37.20,38.00,14.60,11.20
Current/Original Salesforce,96.8,97.2,142.40,52.70,24.10,27.30,29.70,56.80


Next, we will construct a function that computes how profit varies as a function of salesforce size.

In [8]:
def negative_profit(sf_size, c, d, adbudg_min, adbudg_max, current_sales, current_sf, p_marg):
    response = adbudg_min + (adbudg_max - adbudg_min) * ((sf_size / current_sf)**c) / (d + (sf_size / current_sf)**c)
    return -(response * current_sales * p_marg - sf_size * 0.057)

Now let's use `scipy.optimize.minimize` to find the profit-maximizing number of salespersons for each of the eight drugs as well as the maximum profit.

In [36]:
# Initialize array for profit-maximizing number of salespersons for the eight drugs
profit_max_sf = np.zeros((8, 2))

# Set our bounds and initial values
lower_bound = 0
x0 = 80 
bounds_object = optimize.Bounds(lower_bound, np.inf)

# Iterate through each drug and find the optimal salespersons and profit
for i in range(8):
    tuple_values = (df_drug_params.iloc[i][0],df_drug_params.iloc[i][1],df_drug_params.iloc[i][2],df_drug_params.iloc[i][3],margin_sf.iloc[1][i],margin_sf.iloc[2][i],margin_sf.iloc[0][i])
    optimizer_output = optimize.minimize(negative_profit, x0, args=tuple_values, method='trust-constr', bounds=bounds_object, options={'verbose': 1})
    profit_max_sf[i][0] = optimizer_output.x
    profit_max_sf[i][1] = -optimizer_output.fun

df_best_sf = pd.DataFrame(profit_max_sf, columns = ['Optimal salespersons','Maximum profit (in millions)'], index = ['Naprosyn','Nipro','Anaprox','Norinyl','Pironil','Lidex','Synalar','Nasalide'])

`gtol` termination condition is satisfied.
Number of iterations: 24, function evaluations: 28, CG iterations: 13, optimality: 3.79e-11, constraint violation: 0.00e+00, execution time: 0.044 s.
`gtol` termination condition is satisfied.
Number of iterations: 21, function evaluations: 26, CG iterations: 12, optimality: 5.00e-09, constraint violation: 0.00e+00, execution time: 0.038 s.
`gtol` termination condition is satisfied.
Number of iterations: 18, function evaluations: 20, CG iterations: 9, optimality: 6.35e-09, constraint violation: 0.00e+00, execution time: 0.031 s.
`gtol` termination condition is satisfied.
Number of iterations: 15, function evaluations: 14, CG iterations: 6, optimality: 3.59e-09, constraint violation: 0.00e+00, execution time: 0.025 s.
`gtol` termination condition is satisfied.
Number of iterations: 22, function evaluations: 30, CG iterations: 14, optimality: 4.08e-09, constraint violation: 0.00e+00, execution time: 0.033 s.
`gtol` termination condition is satis

Below, we can see our profit-maximizing number of salespersons and maximum profit obtained for each of the eight drugs.

In [37]:
df_best_sf

,Optimal salespersons,Maximum profit (in millions)
Naprosyn,270.512837,221.194475
Nipro,330.045573,228.619794
Anaprox,171.686138,12.951849
Norinyl,71.333349,13.212085
Pironil,36.880309,26.468804
Lidex,50.004889,23.958271
Synalar,30.654988,6.168039
Nasalide,71.160750,3.488045


## Part C:

Now let's compute the profit-maximizing number of salespersons for each of the eight drugs subject to the constraint that only 700 salespersons are available across all eight drugs. We will be doing multivariate optimization here using `scipy.optimize.minimize` once again.

First, let's extract the parameters of our drugs to their own arrays.

In [28]:
c_values = df_drug_params.iloc[:,0].to_numpy()
d_values = df_drug_params.iloc[:,1].to_numpy()
min_values = df_drug_params.iloc[:,2].to_numpy()
max_values = df_drug_params.iloc[:,3].to_numpy()
sales_values = margin_sf.iloc[1,:].to_numpy()
sf_values = margin_sf.iloc[2,:].to_numpy()
margin_values = margin_sf.iloc[0,:].to_numpy()

Next, we need to create a new negative profit function that computes the negative of the profit by taking in the parameters of each of the eight drugs as inputs.

In [41]:
def constrained_profit(sf_size, c, d, adbudg_min, adbudg_max, current_sales, current_sf, p_marg):
    negative_profit = 0
    for i in range(8):
        response = adbudg_min[i] + (adbudg_max[i] - adbudg_min[i]) * ((sf_size[i] / current_sf[i])**c[i]) / (d[i] + (sf_size[i] / current_sf[i])**c[i])
        negative_profit = negative_profit + (-(response * current_sales[i] * p_marg[i] - sf_size[i] * 0.057))
    return negative_profit

Next, let's use the optimize function to find the optimal number of salespersons.

In [81]:
# Initialize array for constrained profit-maximizing number of salespersons for the eight drugs
constrained_sf = np.zeros((8, 4))

# Set our bounds, initial values, and constraints
n_drugs = 8
total_salesforce_size = 700
lower_bound = 0
x0 = np.ones(n_drugs)*total_salesforce_size/n_drugs
sum_constraint_object = optimize.LinearConstraint(np.ones((1,n_drugs)), lower_bound, total_salesforce_size)
bounds_object2 = optimize.Bounds(lower_bound, np.inf)

# Iterate through each drug and find the optimal number of salespersons
optimizer_output2 = optimize.minimize(constrained_profit, x0, args=(c_values, d_values, min_values, max_values, sales_values, sf_values, margin_values), method='SLSQP', bounds=bounds_object, constraints=sum_constraint_object)

From here, we can compute optimal profit for each drug following the new salesperson assignment along with the percentage profit reduction and percentage salesforce reduction.

In [86]:
# New number of sales persons
constrained_sf[:,0] = optimizer_output2.x

# New objective function values
for i in range(8):
    new_profit = -negative_profit(constrained_sf[i][0], df_drug_params.iloc[i][0],df_drug_params.iloc[i][1],df_drug_params.iloc[i][2],df_drug_params.iloc[i][3],margin_sf.iloc[1][i],margin_sf.iloc[2][i],margin_sf.iloc[0][i])
    constrained_sf[i][1] = new_profit

# Percentage profit reductions
for i in range(8):
    constrained_sf[i][2] = ((profit_max_sf[i][1] - constrained_sf[i][1]) / profit_max_sf[i][1]) * 100

# Percentage salesforce reductions
for i in range(8):
    constrained_sf[i][3] = ((profit_max_sf[i][0] - constrained_sf[i][0]) / profit_max_sf[i][0]) * 100
    
df_constrained_sf = pd.DataFrame(constrained_sf, columns = ['Optimal salespersons','Maximum profit (in millions)', 'Percentage profit reduction', 'Percentage salesforce reduction'], index = ['Naprosyn','Nipro','Anaprox','Norinyl','Pironil','Lidex','Synalar','Nasalide'])
df_constrained_sf

,Optimal salespersons,Maximum profit (in millions),Percentage profit reduction,Percentage salesforce reduction
Naprosyn,220.257812,219.549223,0.743803,18.577686
Nipro,245.922451,225.899031,1.190082,25.488335
Anaprox,117.453135,11.068291,14.542773,31.588457
Norinyl,51.530865,12.552384,4.993157,27.760485
Pironil,29.491311,26.226459,0.915584,20.035076
Lidex,35.344425,23.481539,1.989844,29.318062
Synalar,0.000000,4.562604,26.028280,100.000000
Nasalide,0.000000,0.914175,73.791182,100.000000


As we can see above, Synalar and Nasaslide had 100% reductions in their salesforce. This is likely due to the fact that the maximum profit they were originally generating were very low compared to the rest of the drugs. In addition,  drugs like Naprosyn, Nipro, and Pironil had relatively low reductions in their salesforce since they generated more profit overall and also had higher profit margins to begin with.

## Part D:

Lower price elasticity implies that we should advertise more because assuming that advertising elasticity remains steady, we must increase the amount of advertising we do in order to maintaing or increase our sales numbers as price elasticity starts to lower. 

For example, if we take a look at the Dorfman-Steiner Allocation "Rule", we have (Advertising/Sales) = (Advertising Elasticity/-Price Elasticity). If price elasticity is lowered, and we do not increase advertising, than we end up losing sales in order to maintain this rule. So if we'd like to maintain or increase our sales, we must raise our advertising when following the Dorfman-Steiner Allocation "Rule".